# Выявление токсичных комментариев

# Описание проекта 

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо будет обучить модель, которая будет классифицировать комментарии на позитивные и негативные. В распоряжении есть набор данных с разметкой о токсичности правок.

Сервис предоставил датасет `toxic_comments.csv`, в котором столбцы: 
- `text` - содержит текст комментария
- `toxic` - целевой признак

В целом алгоритм решения будет выглядеть следующим образом
1. Выгрузка данных 
2. Предобработка данных
3. EDA
4. Подготовка текста и выборок
5. Обучение моделей
6. Выводы и рекомендации для заказчика

# Выгрузка и ознакомление с данными 

In [1]:
# Импорт необходимых библиотек и надстройки окуржения
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import nltk

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from re import sub
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\79952\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79952\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\79952\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79952\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Преборазование в датафреймы 
data = pd.read_csv('C:/Users/79952/Desktop/Datasets/toxic_comments.csv')

In [3]:
# Общая информация о датасете 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Пропусков в датасете нет. Есть столбец `Unnamed: 0`, который просто дублирует индексы исходной таблицы, удалим его в предобработке

# Предобработка данных 

In [5]:
data = data[['text', 'toxic']]
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
# Проверка на классы toxic
data['toxic'].unique()

array([0, 1], dtype=int64)

In [7]:
# Проверка на явные дубликаты 
data.duplicated().sum()

0

Явных дубликатов не найдено, все значения и типы данных корректны, всего два класса у `toxic`. Предобработка окончена

# Исследовательский анализ

Нет смысла проводить анализ `text`, так как это уникальные категориальные значения. Проведем анализ классов столбца `text`

In [8]:
data['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

Наблюдается явных дизбаланс: положительных комментариев намного больше, чем отрицательных. Следует учесть это при разбиении на выборки

# Подготовка текста 

Подготовим будущий корпус для обучения. Для этого напишем 2 функции по лемматизации и очистки текста от ненужных символов. Учитываем то, что наш текст написан только на английском языке. 

In [9]:
sentence = 'feet striped bats hanging ... '

In [10]:
# Функция перевода в POS тег для лучшей лемматизации
def penn2morphy(penntag):
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

In [11]:
def lemm(text): 
    lemmatizer = WordNetLemmatizer()
    result = [lemmatizer.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(nltk.word_tokenize(text))]
    return ' '.join(result)

In [12]:
lemm(sentence)

'foot strip bat hang ...'

Функция лемматизации готова

In [13]:
# Функция очистки текста
def clear(text): 
    cleared = sub(r'[^a-zA-Z]', ' ' , text).split()
    cleared = ' '.join(cleared)
    return cleared 

In [14]:
clear(sentence)

'feet striped bats hanging'

Функция очистки готова

Создадим новый столбец в датафреме под названием `lemm_text` и применим к нему эти функции

In [15]:
tqdm.pandas()

In [16]:
data['lemm_text'] = data['text'].progress_apply(lemm)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [17]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww ! he match this background colour i 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , i 'm really not try to edit war . it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` more i ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."


In [18]:
data['lemm_text'] = data['lemm_text'].apply(clear)

In [19]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Лемматизированный и очищенный текст готов

# Подготовка выборок 

In [20]:
# Входные/целевой признаки
X = data['lemm_text']
y = data['toxic']

In [21]:
rs = 1

# Разбиение на выборки
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                   y, 
                                                   random_state=rs, 
                                                   stratify=y)

Извлечем признаки из слов при помощи TF-IDF

In [22]:
# Стоп-слова 
stopwords = list(nltk_stopwords.words('english'))

Признаки и выборки готовы, приступим к обучению моделей

# Обучение моделей 

Обучим модель логистической регрессии и метод опорных векторов, использовав кросс валидацию

In [25]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords)), 
    ('models', LinearSVC(random_state=rs)),
])
pipe

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('models', LinearSVC(random_state=1))])

In [26]:
# Подбор скейлеров и моделей
params = [
    {
    'models' : [LinearSVC(random_state=rs)],
    'models__C' : np.array([1, 2, 3]) 
    },
    {
    'models' : [LogisticRegression(solver='liblinear', penalty='l1', random_state=rs)],
    'models__C' : np.array([1, 2, 3])
    }
]

In [27]:
grid = GridSearchCV( 
        estimator=pipe, 
        param_grid=params, 
        cv=5, 
        scoring= 'f1', 
        n_jobs=-1, 
        verbose=3)

In [28]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('models', LinearSVC(random_state=1))]),
             n_jobs=-1,
             param_grid=[{'models': [LinearSVC(random_state=1)],
                          'models__C': array([1, 2, 3])},
                         {'models': [LogisticRegression(penalty='l1',
                                                        random_state=1,
                                                        solver='liblinear')],
                          'models__C': array([1, 2, 3])}],
             scoring='f1', verbose=3)

In [29]:
grid.best_params_

{'models': LogisticRegression(penalty='l1', random_state=1, solver='liblinear'),
 'models__C': 3}

In [30]:
print(f'Наилучший показатель метрики при кросс валидации: {grid.best_score_}')

Наилучший показатель метрики при кросс валидации: 0.7795759284877075


Получим предсказания и посчитаем метрику на тестовой выборке

In [32]:
predictions = grid.best_estimator_.predict(X_test)

In [33]:
print(f'Метрика на тестовой выборке {f1_score(y_test, predictions)}')

Метрика на тестовой выборке 0.7788474576271186


Метрика показала результат больше 0,75 данная модель с предложенными параметрами подходит для решения задачи

# Общий вывод 

В данном отчете была проведена следующая работа: 
1. Выгрузка и ознакомление с данными 
2. Предобработка данных
3. Исследовательский анализ 
4. Подготовка текста, а именно его лемматизация 
5. Подготовка выборок для обучения модели
6. Обучение логистической регрессии

Как итог, можно сделать вывод о том, что логистическая регрессия с приведенными ниже гиперпараметрами вполне подходит для решения данной задачи, так как показала значение метрики F1 более 0.75

In [34]:
grid.best_params_

{'models': LogisticRegression(penalty='l1', random_state=1, solver='liblinear'),
 'models__C': 3}

Заказчику рекомендуется выбрать данную модель для классификации комментариев на позитивные и негативные